# Данное задание основано на материалах лекций по метрическим методам и посвящено подбору числа соседей в методе kNN.

В этом задании нужно подобрать оптимальное значение k для алгоритма kNN. Будем использовать набор данных Wine, где требуется предсказать сорт винограда, из которого изготовлено вино, используя результаты химических анализов.

In [1]:
import pandas as pd
data = pd.read_csv('wine.data')

In [10]:
data.head(5)

,Class,Alcohol,MalicAcid,Ash,AlcOfAsh,Magnesium,TotalPhenols,Flavanoids,NonflavanoidPhenols,Proanthocyanins,ColorIntensity,Hue,DilutedWines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [11]:
from sklearn.model_selection import KFold

In [19]:
X = data[data.columns[1:14]]
X.head(5)

,Alcohol,MalicAcid,Ash,AlcOfAsh,Magnesium,TotalPhenols,Flavanoids,NonflavanoidPhenols,Proanthocyanins,ColorIntensity,Hue,DilutedWines,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


Выделим целевую переменную

In [16]:
y = data['Class']
y.head(5)

0    1
1    1
2    1
3    1
4    1
Name: Class, dtype: int64

Оценку качества проводим методом кросс-валидации по 5 блокам (5-fold). 
Создаем генератор разбиений, который перемешивает выборку перед формированием блоков (shuffle=True). 
В качестве меры качества использована доля верных ответов (accuracy).
Ищем точность классификации на кросс-валидации для метода k ближайших соседей (sklearn.neighbors.KNeighborsClassifier), при k от 1 до 50. При каком k получилось оптимальное качество? Чему оно равно (число в интервале от 0 до 1)? 

In [27]:
from sklearn.model_selection import cross_val_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
#for train_indices, test_indices in kf.split(X):
    #print('Train: %s | test: %s' % (train_indices, test_indices))
#results = cross_val_score(estimator, X, y, cv=kf)


In [43]:
from sklearn.neighbors import KNeighborsClassifier
array=[]
k=1
while k<51:
    neigh = KNeighborsClassifier(n_neighbors=k)
    results = cross_val_score(neigh, X, y, cv=kf, scoring='accuracy')
   # print results.mean()
    array.append(results.mean())
    k = k+1
max_res = max(array)
print max_res
array.index(max_res)

0.7304761904761905


0

Заметим, что при использовании сырых данных, k=0, что свидетельствует о том, что метрические методы чувствительны к масштабу признаков — так, если масштаб одного из признаков существенно превосходит масштабы остальных признаков, то их значения практически не будут влиять на ответы алгоритма. Поэтому важно производить масштабирование признаков. Обычно это делается путем вычитания среднего значения признака и деления на стандартное отклонение.

In [76]:
from sklearn import preprocessing
X_scaled = preprocessing.scale(X)

In [47]:
array=[]
k=1
while k<51:
    neigh = KNeighborsClassifier(n_neighbors=k)
    results = cross_val_score(neigh, X_scaled, y, cv=kf, scoring='accuracy')
   # print results.mean()
    array.append(results.mean())
    k = k+1
max_res = max(array)
print max_res
array.index(max_res)+1

0.9776190476190475


29

# Boston houses (metrics)

Данное задание основано на лекциях по метрическим методам и посвящено выбору наилучшей метрики для конкретной задачи.

Мы будем использовать в данном задании набор данных Boston, где нужно предсказать стоимость жилья на основе различных характеристик расположения (загрязненность воздуха, близость к дорогам и т.д.). 

Главным параметром любого метрического алгоритма является функция расстояния (или метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более эффективным вариантом является подбор метрики под конкретную задачу. Один из подходов — использование той же евклидовой метрики, но с весами: каждой координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние. Веса настраиваются с целью оптимизации качества на отложенной выборке. Другой подход, о котором и пойдет речь в данном задании — выбор метрики из некоторого класса метрик. Мы возьмем за основу метрику Минковского.

In [51]:
from sklearn.datasets import load_boston
df = load_boston()

In [56]:
df.data

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
        4.9800e+00],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        9.1400e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
        4.0300e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        5.6400e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
        6.4800e+00],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
        7.8800e+00]])

In [74]:
from sklearn import preprocessing
df_ = preprocessing.scale(df.data)
y = df.target
df_

array([[-0.41771335,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41526932, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41527165, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41137448, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40568883, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41292893, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [77]:
from sklearn.model_selection import KFold
kf_ = KFold(n_splits=5, shuffle=True, random_state=42)

Переберем разные варианты параметра метрики p по сетке от 1 до 10 с таким шагом, чтобы всего было протестировано 200 вариантов (функция numpy.linspace). Используем KNeighborsRegressor с n_neighbors=5 и weights='distance' — данный параметр добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. В качестве метрики качества используем среднеквадратичную ошибку (параметр scoring='mean_squared_error' у cross_val_score.

In [78]:
import numpy as np
p_array = np.linspace(1, 10, 200)

In [81]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
array_ = []
for i in p_array:
    neigh_ = KNeighborsRegressor(n_neighbors=5, weights='distance', p=i, metric='minkowski')
    results_ = cross_val_score(neigh_, df_, y, cv=kf_, scoring='neg_mean_squared_error')
    array_.append(results_.mean())
    


In [83]:
max_res = max(array_)
max_res

-16.050208508436157

In [84]:
array.index(max_res)+1

1

# Perceptron (linear model) 

Суть задания: повысить качество лин модели путем нормализации признаков.

In [6]:
import pandas as pd
train_data = pd.read_csv("perceptron-train.csv")
test_data = pd.read_csv("perceptron-test.csv")

In [9]:
test_data.head(5)

,Class,Feature_1,Feature_2
0,-1.0,1.651437,1337.453826
1,1.0,-0.866495,1191.232457
2,-1.0,0.789828,-475.647768
3,-1.0,0.179549,1959.095353
4,1.0,-0.434351,568.504207


In [50]:
from sklearn.linear_model import Perceptron
clf = Perceptron(random_state = 241)
y = train_data['Class']
X_train = train_data[train_data.columns[1:3]]
clf.fit(X_train, y)


Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=241,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [51]:
X_test = test_data[test_data.columns[1:3]]
predictions = clf.predict(X_test)
y_test = test_data['Class']

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.655

In [53]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [54]:
clf.fit(X_train_scaled, y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      max_iter=None, n_iter=None, n_jobs=1, penalty=None, random_state=241,
      shuffle=True, tol=None, verbose=0, warm_start=False)

In [55]:
predictions_scaled = clf.predict(X_test_scaled)

In [56]:
accuracy_score(y_test, predictions_scaled)

0.845

In [57]:
acc1 = accuracy_score(y_test, predictions) 
acc2 = accuracy_score(y_test, predictions_scaled)

In [58]:
acc = acc2-acc1
acc

0.18999999999999995